In [1]:
import pandas as pd
from tabulate import tabulate
from scipy.stats import fisher_exact
from scipy.stats import chi2_contingency

/Users/huyanshen/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
data =  pd.read_excel('/Users/huyanshen/Desktop/4-8-table/AsiaVsUs-shh.xlsx', header=0)

In [3]:
data_shh = data[data['Genetic Mutations'].isin(['TP53+', 'TP53-'])]

In [4]:
intrst_list = ['Sex', 'Race', 'Tumor Texture', 'Histological Subtypes', 'TumorResection', 'Radiotherapy', 'Chemotherapy', 'Survival Status']

In [5]:
for cato in intrst_list:
    contig = pd.crosstab(data_shh[cato], data_shh['Genetic Mutations'])
    print(tabulate(contig, headers='keys', tablefmt='fancy_grid'))
    chi2, p_value, dof, expected = chi2_contingency(contig, correction=True)
    print("P-value:", p_value)

╒═══════╤═════════╤═════════╕
│ Sex   │   TP53+ │   TP53- │
╞═══════╪═════════╪═════════╡
│ F     │      21 │      47 │
├───────┼─────────┼─────────┤
│ M     │      39 │     103 │
╘═══════╧═════════╧═════════╛
P-value: 0.7265175258732204
╒═══════════════════════╤═════════╤═════════╕
│ Race                  │   TP53+ │   TP53- │
╞═══════════════════════╪═════════╪═════════╡
│ Asian                 │      57 │     137 │
├───────────────────────┼─────────┼─────────┤
│ Other or not reported │       3 │       8 │
├───────────────────────┼─────────┼─────────┤
│ White                 │       0 │       5 │
╘═══════════════════════╧═════════╧═════════╛
P-value: 0.35496679767535483
╒═════════════════╤═════════╤═════════╕
│ Tumor Texture   │   TP53+ │   TP53- │
╞═════════════════╪═════════╪═════════╡
│ Hard            │      11 │      28 │
├─────────────────┼─────────┼─────────┤
│ Mixed           │      13 │      36 │
├─────────────────┼─────────┼─────────┤
│ Soft            │      31 │      72 │

In [11]:
from lifelines import KaplanMeierFitter
mutations = ['TP53+', 'TP53-', ]
for mutation in mutations:
    print(f'For {mutation}')
    kmf = KaplanMeierFitter()
    kmf.fit(durations=data_shh[data_shh['Genetic Mutations'] == mutation]['OS-impute (months)'], event_observed=data_shh[data_shh['Genetic Mutations'] == mutation]['Survival Status'])
    time_points = [12, 24, 36, 48, 60]

    # Get survival probability at the specified time point
    for time_point in time_points:
        survival_prob = kmf.survival_function_at_times([time_point])
        print(f"Survival probability at {time_point} months:", survival_prob)

print("for all:")
kmf = KaplanMeierFitter()
kmf.fit(durations=data_shh['OS-impute (months)'], event_observed=data_shh['Survival Status'])
time_points = [12, 24, 36, 48, 60]

# Get survival probability at the specified time point
for time_point in time_points:
    survival_prob = kmf.survival_function_at_times([time_point])
    print(f"Survival probability at {time_point} months:", survival_prob)


For TP53+
Survival probability at 12 months: 12    0.88474
Name: KM_estimate, dtype: float64
Survival probability at 24 months: 24    0.740054
Name: KM_estimate, dtype: float64
Survival probability at 36 months: 36    0.549754
Name: KM_estimate, dtype: float64
Survival probability at 48 months: 48    0.549754
Name: KM_estimate, dtype: float64
Survival probability at 60 months: 60    0.366503
Name: KM_estimate, dtype: float64
For TP53-
Survival probability at 12 months: 12    0.916764
Name: KM_estimate, dtype: float64
Survival probability at 24 months: 24    0.812814
Name: KM_estimate, dtype: float64
Survival probability at 36 months: 36    0.766846
Name: KM_estimate, dtype: float64
Survival probability at 48 months: 48    0.697983
Name: KM_estimate, dtype: float64
Survival probability at 60 months: 60    0.697983
Name: KM_estimate, dtype: float64
for all:
Survival probability at 12 months: 12    0.906826
Name: KM_estimate, dtype: float64
Survival probability at 24 months: 24    0.79157

In [7]:
import pandas as pd
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test

kmf = KaplanMeierFitter()
groups = data_shh['Genetic Mutations']
time = data_shh['OS-impute (months)']
event = data_shh['Survival Status']

# Perform log-rank test
results = multivariate_logrank_test(time, groups, event)

# Print the results (p-value)
p_value = results.p_value
print("Overall comparison (Multivariate Log-rank test) p-value:", p_value)

Overall comparison (Multivariate Log-rank test) p-value: 0.06847405240200605
